In [14]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals


In [15]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [16]:
(x_train_ex, y_train_ex), (x_test_ex, y_test_ex) = tf.keras.datasets.mnist.load_data()

In [17]:
y_train_ex

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [18]:
#reshapes samples into column vectors + converts to grayscale
x_train = x_train.astype(np.float32).reshape(-1,28*28) / 255.0 
x_test = x_test.astype(np.float32).reshape(-1,28*28) / 255.0


In [19]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#letting last 10000 samples be validation set 
x_validation, x_train = x_train[50000:], x_train[:50000]
y_validation, y_train = y_train[50000:], y_train[:50000]


In [20]:
#model constants 
n_columns = 28*28
n_hidden1 = 16 #layer size
n_hidden2 = 10
num_outputs = 10
learning_rate = 0.01


In [21]:
X = tf.placeholder(tf.float32, shape=(None, n_columns), name="X")
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

In [22]:
 

stddev = 2 / np.sqrt(n_columns)
w1_init = tf.truncated_normal((n_columns,n_hidden1),stddev=stddev)
w1 = tf.Variable(w1_init, name = 'w1')
b1 = tf.Variable(tf.zeros([n_hidden1]), name = 'b1')
logit1 = tf.matmul(X, w1) + b1
h1 = tf.nn.relu(logit1)
#h1 should have shape batchsize, 16

stddev2 = 2 / np.sqrt(n_hidden1)
w2_init = tf.truncated_normal((n_hidden1,n_hidden2),stddev=stddev2)
w2 = tf.Variable(w2_init, name = 'w2')
b2 = tf.Variable(tf.zeros([n_hidden2]), name = 'b2')
logit2 = tf.matmul(h1, w2) + b2
h2 = tf.nn.relu(logit2)

stddev3 = 2 / np.sqrt(int(h2.get_shape()[1]))
w3_init = tf.truncated_normal((n_hidden2,num_outputs), stddev=stddev3)
w3 = tf.Variable(w3_init, name = 'w3')
b3 = tf.Variable(tf.zeros([num_outputs]), name = 'b3')
final_logits = tf.matmul(h2,w3) + b3


x_entrop = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=final_logits)
loss = tf.reduce_mean(x_entrop, name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)


correct = tf.nn.in_top_k(final_logits, Y, 1) #????
acc = tf.reduce_mean(tf.cast(correct, tf.float32)) #????


In [23]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [24]:
init = tf.global_variables_initializer()

In [28]:
n_epochs = 50
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train_op, feed_dict={X:x_batch, Y: y_batch})
            
        acc_batch = acc.eval(feed_dict={X:x_batch, Y:y_batch})
        acc_val = acc.eval(feed_dict={X:x_validation, Y:y_validation})
        print(epoch, "Batch acc:", acc_batch, "Val acc:", acc_val)

0 Batch acc: 0.8 Val acc: 0.8244
1 Batch acc: 0.96 Val acc: 0.8916
2 Batch acc: 0.92 Val acc: 0.9068
3 Batch acc: 0.94 Val acc: 0.9129
4 Batch acc: 0.92 Val acc: 0.9179
5 Batch acc: 0.9 Val acc: 0.9228
6 Batch acc: 0.94 Val acc: 0.9279
7 Batch acc: 0.9 Val acc: 0.9314
8 Batch acc: 0.92 Val acc: 0.9335
9 Batch acc: 0.88 Val acc: 0.9351
10 Batch acc: 0.96 Val acc: 0.9386
11 Batch acc: 0.98 Val acc: 0.9383
12 Batch acc: 0.98 Val acc: 0.9393
13 Batch acc: 0.94 Val acc: 0.9426
14 Batch acc: 0.94 Val acc: 0.9434
15 Batch acc: 1.0 Val acc: 0.9442
16 Batch acc: 0.96 Val acc: 0.945
17 Batch acc: 0.98 Val acc: 0.9468
18 Batch acc: 0.98 Val acc: 0.948
19 Batch acc: 0.96 Val acc: 0.9482
20 Batch acc: 0.88 Val acc: 0.9478
21 Batch acc: 0.98 Val acc: 0.9494
22 Batch acc: 0.96 Val acc: 0.9489
23 Batch acc: 0.98 Val acc: 0.9491
24 Batch acc: 0.96 Val acc: 0.9498
25 Batch acc: 0.98 Val acc: 0.9498
26 Batch acc: 0.94 Val acc: 0.9518
27 Batch acc: 0.94 Val acc: 0.9518
28 Batch acc: 1.0 Val acc: 0.9506
29

KeyboardInterrupt: 